In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [2]:
import keras
from deel.lip.layers import (
    SpectralDense,
    SpectralConv2D,
    ScaledL2NormPooling2D,
    FrobeniusDense,
)
from keras.models import Sequential
# from deel.lip.model import Sequential
from deel.lip.activations import GroupSort
from deel.lip.losses import MulticlassHKR, MulticlassKR, HKR, HingeMargin
from keras.layers import Input, Flatten, Dense
from keras.optimizers import Adam
from keras.datasets import mnist
from keras.utils import to_categorical
import numpy as np
import keras.ops as K
from keras.utils import to_categorical
import matplotlib.pyplot as plt

In [3]:
# first we select the two classes
selected_classes = [0, 8]  # must be two classes as we perform binary classification


def prepare_data(x, y, class_a=0, class_b=8):
    """
    This function convert the MNIST data to make it suitable for our binary classification
    setup.
    """
    # select items from the two selected classes
    mask = (y == class_a) + (
        y == class_b
    )  # mask to select only items from class_a or class_b
    x = x[mask]
    y = y[mask]
    x = x.astype("float32")
    y = y.astype("float32")
    # convert from range int[0,255] to float32[-1,1]
    x /= 255
    x = x.reshape((-1, 28, 28, 1))
    # change label to binary classification {-1,1}
    y[y == class_a] = 1.0
    y[y == class_b] = 0.0
    return x, y.reshape((-1, 1))


# now we load the dataset
(x_train, y_train_ord), (x_test, y_test_ord) = mnist.load_data()
# prepare the data
x_train, y_train = prepare_data(
    x_train, y_train_ord, selected_classes[0], selected_classes[1]
)
x_test, y_test = prepare_data(
    x_test, y_test_ord, selected_classes[0], selected_classes[1]
)
y_test_ord = y_test[:,0]
y_train_ord = y_train[:,0]
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)

In [4]:
x_train = np.transpose(x_train,(0,3,1,2))
x_test = np.transpose(x_test,(0,3,1,2))

In [5]:
vanilla_model_1 = keras.models.load_model("/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output.keras")
vanilla_model_1.compile(
   
    loss=HKR(
        alpha=10.0, min_margin=1.0
    ),  # HKR stands for the hinge regularized KR loss
    metrics=[
        # KR,  # shows the KR term of the loss
        HingeMargin(min_margin=1.0),  # shows the hinge term of the loss
    ],
    optimizer=Adam(learning_rate=0.001),)
vanilla_model_1.summary()

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 12 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "lipModel"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense (Dense)          │ (None, 32)             │        25,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense_1 (Dense)        │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense_2 (Dense)        │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,664 (100.25 KB)

 Trainable params: 25,664 (100.25 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
vanilla_model_2 = keras.models.load_model("/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output_converted_2.keras")
vanilla_model_2.compile(
        # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
        # note also in the case of lipschitz networks, more robustness require more parameters.
        loss=MulticlassHKR(alpha=100, min_margin=0.25),
        optimizer=Adam(1e-4),
        metrics=["accuracy", MulticlassKR()],)
vanilla_model_2.summary()

Model: "reflected_two_logits_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 1, 28, 28) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 784)       │          0 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spectral_dense      │ (None, 32)        │     25,120 │ flatten[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spectral_dense_1    │ (None, 16)        │        528 │ spectral_dense[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 3)         │         51 │ spectral_dense_1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 1)         │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ negative (Negative) │ (None, 1)         │          0 │ get_item[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 2)         │          0 │ dense[0][0]       │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reflected_two_logi… │ (None, 4)         │          0 │ negative[0][0],   │
│ (Concatenate)       │                   │            │ get_item[0][0],   │
│                     │                   │            │ get_item_1[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 25,699 (100.39 KB)

 Trainable params: 25,699 (100.39 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
vanilla_model_4 = keras.models.load_model("/home/aws_install/robustess_project/lip_models/demo3_FC_vanilla_MNIST08_channelfirst_False_disj_Neurons_single_output_converted_2_4_logits.keras")
vanilla_model_4.compile(
        # decreasing alpha and increasing min_margin improve robustness (at the cost of accuracy)
        # note also in the case of lipschitz networks, more robustness require more parameters.
        loss=MulticlassHKR(alpha=100, min_margin=0.25),
        optimizer=Adam(1e-4),
        metrics=["accuracy", MulticlassKR()],)
vanilla_model_4.summary()

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense (Dense)          │ (None, 32)             │        25,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spectral_dense_1 (Dense)        │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,699 (100.39 KB)

 Trainable params: 25,699 (100.39 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# strategy: first
# we select a sample from each class.
images_list = []
labels_list = []
# select only a few element from the test set
# selected = np.random.choice(len(y_test_ord), 500)
sub_y_test_ord = y_test_ord[:400]
sub_x_test = x_test[:400]
# drop misclassified elements
misclassified_mask = K.equal(
    K.argmax(vanilla_model_2.predict(sub_x_test, verbose=0), axis=-1), sub_y_test_ord
)
sub_x_test = sub_x_test[misclassified_mask.detach().cpu().numpy()]
sub_y_test_ord = sub_y_test_ord[misclassified_mask.detach().cpu().numpy()]
for i in range(2):
    # select the 20 firsts elements of the ith label
    label_mask = sub_y_test_ord == i
    x = sub_x_test[label_mask][:100]
    y = sub_y_test_ord[label_mask][:100]
    # convert it to tensor for use with foolbox
    images = K.convert_to_tensor(x.astype("float32"), dtype="float32")
    labels = K.convert_to_tensor(y, dtype="int64")
    # repeat the input 10 times, one per misclassification target
    for j in range(100):
        images_list.append(images[j])
        labels_list.append(labels[j])
images = K.convert_to_tensor(images_list)
labels = K.convert_to_tensor(labels_list)
labels.shape

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer_1']
Received: inputs=Tensor(shape=torch.Size([32, 1, 28, 28]))
  warnings.warn(msg)
/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer_1']
Received: inputs=Tensor(shape=torch.Size([16, 1, 28, 28]))
  warnings.warn(msg)


torch.Size([200])

In [9]:
def compute_binary_certificate(images, model, L=1):    
    values = model(images)[:,0]
    certificates = np.abs(values.detach().cpu().numpy())/L
    return certificates   

def compute_certificate(images, model, L=1):    
    values, _ = K.top_k(model(images), k=2)
    certificates = (values[:, 0] - values[:, 1]) / (np.sqrt(2)*L)
    return certificates.detach().cpu().numpy()   

In [10]:
lip_radius = compute_binary_certificate(images, vanilla_model_1)

In [11]:
lip = lip_radius = compute_certificate(images, vanilla_model_2)

/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer_1']
Received: inputs=Tensor(shape=torch.Size([200, 1, 28, 28]))
  warnings.warn(msg)


In [12]:
vanilla_model_2.predict(images[:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


/home/aws_install/miniconda3/envs/k3torchenv/lib/python3.10/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['input_layer_1']
Received: inputs=Tensor(shape=torch.Size([1, 1, 28, 28]))
  warnings.warn(msg)


array([[ 1.861982e+00, -1.861982e+00, -1.000000e+04, -1.000000e+04]],
      dtype=float32)

In [13]:
vanilla_model_1.predict(images[:1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[-1.861982]], dtype=float32)